# 搭建yolo v3
注：该文件仅记录搭建网络的步骤，全部用markdown写成
> [YOLOv3论文解析: An Incremental Improvement](https://blog.csdn.net/mieleizhi0522/article/details/79919875)
## 配置文件
配置文件保存了yolo网络的全部网络参数，保存于/cfg/yolov3.cfg中

在cgf文件中可以看到，YOLO共用了五种层结构：
### Convolutional
```cfg
[convolutional]
batch_normalize=1 
filters=64 
size=3 
stride=1 
pad=1 
activation=leaky
```
除上面代码提到的外，还有另外两个参数不同的卷积层结构。
### Shortcut
```cfg
[shortcut]
from=-3 
activation=linear 
```
shortcut层是一个跳过连接操作，在残差神经网络ResNet中有出现，具体可见吴恩达的深度学习课程，我后面也会写一篇讲ResNet的文章（又在给自己挖坑了😑）
### Upsample
```cfg
[upsample]
stride=2
```
用以双线性上采样
### Route
```cfg
[route]
layers = -4

[route]
layers = -1, 61
```
路由层，它的参数 layers 有一个或两个值。
- 仅有一个值时，输出这一层通过该值索引的特征图，这里设置为-4，所以层级将输出路由层之前第四个层的特征图。
- 当层级有两个值时，它将返回由这两个值索引的拼接特征图，这里设置为为-1 和 61，因此该层级将输出从前一层级（-1）到第 61 层的特征图，并将它们按深度拼接。

### YOLO
```cfg
[yolo]
mask = 0,1,2
anchors = 10,13, 16,30, 33,23, 30,61, 62,45, 59,119, 116,90, 156,198, 373,326
classes=80
num=9
jitter=.3
ignore_thresh = .5
truth_thresh = 1
random=1
```
yolo层级用于最后的目标检测。参数 anchors 定义了 9 组锚点，但是它们只是由 mask 标签使用的属性所索引的锚点。这里，mask 的值为 0、1、2 表示了第一个、第二个和第三个使用的锚点，表示每一个单元格预测三个边框。总而言之，预测层的规模为3，共9个锚点。
### Net
还有一个部分net，只描述网络输入和训练参数的相关信息，并未用于 YOLO 的前向传播，所以并不计算为层。但这一部分它为我们提供了网络输入大小等信息，可用于调整前向传播中的锚点。
```cfg
[net]
# Testing
batch=1
subdivisions=1
# Training
# batch=64
# subdivisions=16
width= 320
height = 320
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1
```

## 解析配置文件
需要创建一个darknet.py文件，Darknet是构建 YOLO 底层架构的环境（这个架构同样也由该作者开发），这个文件将包含实现 YOLO 网络的所有代码。同样我们还需要补充一个名为 util.py 的文件，它会包含多种需要调用的函数。在将所有这些文件保存在检测器文件夹下后，我们就能使用 git 追踪它们的改变。

在darknet.py 文件顶部添加必要的导入项。
```python
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
```
我们定义一个函数 parse_cfg，该函数使用配置文件的路径作为输入。
```python
def parse_cfg(cfgfile):
```
该函数用来解析cfg，将每个块存储为词典，最后返回一个列表blocks，列表的元素是字典，每一个字典包含了前面config文件中每一块的信息，如下图：

![](https://s2.ax1x.com/2020/02/05/1r096x.png)

## 创建我们的模型
接下来我们用`creat_module()`函数构建我们的模型，它接受`parse_config`生成的blocks，并返回一个nn.ModuleList()类的对象ModuleList
### 知识点补充：批归一化
从字面意思看来Batch Normalization（简称BN）就是对每一批数据进行归一化，确实如此，对于训练中某一个batch的数据{x1,x2,...,xn}，注意这个数据是可以输入也可以是网络中间的某一层输出。在BN出现之前，我们的归一化操作一般都在数据输入层，对输入的数据进行求均值以及求方差做归一化，但是BN的出现打破了这一个规定，我们可以在网络中任意一层进行归一化处理，因为我们现在所用的优化方法大多都是min-batch SGD，所以我们的归一化操作就成为Batch Normalization。

**我们为什么需要BN？**我们知道网络一旦train起来，那么参数就要发生更新，除了输入层的数据外(因为输入层数据，我们已经人为的为每个样本归一化)，后面网络每一层的输入数据分布是一直在发生变化的，因为在训练的时候，前面层训练参数的更新将导致后面层输入数据分布的变化。以网络第二层为例：网络的第二层输入，是由第一层的参数和input计算得到的，而第一层的参数在整个训练过程中一直在变化，因此必然会引起后面每一层输入数据分布的改变。我们把网络中间层在训练过程中，数据分布的改变称之为：“Internal Covariate Shift”。BN的提出，就是要解决在训练过程中，中间层数据分布发生改变的情况。

**BN怎么做？**
BN步骤主要分为4步：

1. 求每一个训练批次数据的均值
2. 求每一个训练批次数据的方差
3. 使用求得的均值和方差对该批次的训练数据做归一化，获得0-1分布。其中ε是为了避免除数为0时所使用的微小正数。
4. 尺度变换和偏移：将$x_i$乘以$γ$调整数值大小，再加上$β$增加偏移后得到$y_i$，这里的$γ$是尺度因子，$β$是平移因子。这一步是BN的精髓，由于归一化后的$x_i$基本会被限制在正态分布下，使得网络的表达能力下降。为解决该问题，我们引入两个新的参数：$γ$,$β$。 $γ$和$β$是在训练时网络自己学习得到的。

![](https://s2.ax1x.com/2020/02/05/1roZ1P.png)

[【深度学习】批归一化（Batch Normalization）](https://www.cnblogs.com/skyfsm/p/8453498.html)

### 路由层
值得注意的是，我们在导入路由层的时候创建了一个空的层：`route=EmptyLayer()`
```python
elif (x['type'] == 'route'):
    x['layers'] = x['layers'].split(',')
    start=int(x['layers'][0])
    try:
        end=int(x['layers'][0])
    except:
        end=0
    if start>0:
        start=start-index
    if end > 0:  
        end = end - index
    route=EmptyLayer()
    module.add_module("route_{0}".format(index), route)
```
`EmptyLayer`类定义如下：
```python
class EmptyLayer(nn.Module):
    def __init__(self):
        super(EmptyLayer, self).__init__()
```
**为什么要定义一个空的层呢？**

对于在 Route 模块中设计一个层，我们必须建立一个 nn.Module 对象，其作为 layers 的成员被初始化。然后，我们可以写下代码，将 forward 函数中的特征图拼接起来并向前馈送。最后，我们执行网络的某个 forward 函数的这个层。

但拼接操作的代码相当地短和简单（在特征图上调用 torch.cat），所以没有必要在这里直接写上。取而代之，我们可以将一个假的层置于之前提出的路由层的位置上，然后直接在代表 darknet 的 nn.Module 对象的 forward 函数中执行**拼接运算。**

捷径层也使用空的层，因为它还要执行一个非常简单的操作（加）。没必要更新 filters 变量，因为它只是将前一层的特征图添加到后面的层上而已。
### 创建模型的思路
这个yolo复现代码添加网络的思路是：
1. 在一个for的大循环中：
2. 先用`nn.Sequential()`创建对象module
3. 根据层的名称不同用`module.add_module()`添加每一层
4. 每个循环结束时将module添加到module_list上
5. 返回net_info(网络的信息), module_list

## 实现网络的前向传播
前面说过，yolov3是基于Darknet框架，所应定义一个类为`Darknet`：
```python
class Darknet((nn.Module)):
    def __init__(self,cgffile):
        super(Darknet, self ).__init__()
        self.blocks = parse_cfg(cfgfile)
        self.net_info, self.module_list = creat_modules(self.blocks)

    def forward(self,x,CUDA):
```
通过重写forward方法实现yolov3网络的前向传播，要知道，这个网络总共有53层，包含了卷积层和残差块，而且在预测bounding boxes时采用了**跨尺度的预测**（Predictions Across Scales）
![](https://s2.ax1x.com/2020/02/06/1ytP0g.png)